In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
#    for filename in filenames:
#       print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

Common Includes

In [ ]:
import tensorflow as tf
from tensorflow import keras
print("TF version:", tf.__version__)
print("GPU is", "available" if tf.config.list_physical_devices('GPU') else "NOT AVAILABLE")
!lscpu | grep 'Model name'
!lscpu | grep 'Socket(s):'
!lscpu | grep 'Core(s) per socket'
!lscpu | grep 'Thread(s) per core'
!nvidia-smi

Dataset

In [19]:
AUTOTUNE = tf.data.AUTOTUNE
BATCH_SIZE = 32

ds_train = tf.keras.utils.image_dataset_from_directory(
    directory = "../input/imagenetmini-1000/imagenet-mini/train",
    labels = "inferred",
    label_mode = "int",
    color_mode = "rgb",
    batch_size = BATCH_SIZE,
    image_size = (299, 299),
)

ds_val = tf.keras.utils.image_dataset_from_directory(
    directory = "../input/imagenetmini-1000/imagenet-mini/val",
    labels = "inferred",
    label_mode = "int",
    color_mode = "rgb",
    batch_size = BATCH_SIZE,
    image_size = (299, 299)
)

def normalize_image(image, label):
    #image = tf.keras.applications.resnet.preprocess_input(image)
    image = tf.cast(x=image, dtype=tf.float32)/127.5 - 1.0
    return image, label

#ds_train = ds_train.shuffle(100)
ds_train = ds_train.map(map_func=normalize_image, num_parallel_calls=AUTOTUNE)
#ds_train = ds_train.cache()
#ds_train = ds_train.prefetch(buffer_size=AUTOTUNE)

ds_val = ds_val.map(map_func=normalize_image, num_parallel_calls=AUTOTUNE)
ds_val = ds_val.prefetch(buffer_size=AUTOTUNE)

Found 34745 files belonging to 1000 classes.
Found 3923 files belonging to 1000 classes.


Model ResNet152 with 299,299,3 input size

In [ ]:
INPUT_SHAPE = (299,299,3)

model = tf.keras.applications.resnet.ResNet152(
    include_top=True,
    weights=None,
    input_tensor=None,
    input_shape=INPUT_SHAPE,
    classes=1000,
    classifier_activation=None,
)


model.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=keras.optimizers.Adam(),
    metrics=['accuracy', tf.keras.metrics.SparseTopKCategoricalAccuracy(k=5)]
)

model.summary()

In [ ]:
import re
from keras.models import Model

def insert_layer_nonseq(model, layer_regex, insert_layer_factory,
                        insert_layer_name=None, position='after'):

    # Auxiliary dictionary to describe the network graph
    network_dict = {'input_layers_of': {}, 'new_output_tensor_of': {}}

    # Set the input layers of each layer
    for layer in model.layers:
        for node in layer._outbound_nodes:
            layer_name = node.outbound_layer.name
            if layer_name not in network_dict['input_layers_of']:
                network_dict['input_layers_of'].update(
                        {layer_name: [layer.name]})
            else:
                network_dict['input_layers_of'][layer_name].append(layer.name)

    # Set the output tensor of the input layer
    network_dict['new_output_tensor_of'].update(
            {model.layers[0].name: model.input})

    # Iterate over all layers after the input
    model_outputs = []
    for layer in model.layers[1:]:

        # Determine input tensors
        layer_input = [network_dict['new_output_tensor_of'][layer_aux] 
                for layer_aux in network_dict['input_layers_of'][layer.name]]
        if len(layer_input) == 1:
            layer_input = layer_input[0]

        # Insert layer if name matches the regular expression
        if re.match(layer_regex, layer.name):
            if position == 'replace':
                x = layer_input
            elif position == 'after':
                x = layer(layer_input)
            elif position == 'before':
                pass
            else:
                raise ValueError('position must be: before, after or replace')

            new_layer = insert_layer_factory()
            #f insert_layer_name:
            #   new_layer.name = insert_layer_name
            #lse:
            #   new_layer.name = '{}_{}'.format(layer.name, 
            #                                 new_layer.name)
            x = new_layer(x)
            print('New layer: {} Old layer: {} Type: {}'.format(new_layer.name,
                                                            layer.name, position))
            if position == 'before':
                x = layer(x)
        else:
            x = layer(layer_input)

        # Set new output tensor (the original one, or the one of the inserted
        # layer)
        network_dict['new_output_tensor_of'].update({layer.name: x})

        # Save tensor in output list if it is output in initial model
        if layer.name in model.output_names:
            model_outputs.append(x)
            print(x)
            print(model.output_names)
    return Model(inputs=model.inputs, outputs=model_outputs)

In [ ]:
def Avg_pool_2_factory():
    return tf.keras.layers.AveragePooling2D(pool_size=(2, 2), name="avg_pool_2")

def Global_avg_pool_factory():
    return tf.keras.layers.GlobalAveragePooling2D(name="avg_pool_5")

model = insert_layer_nonseq(model=model, layer_regex='avg_pool', insert_layer_factory=Avg_pool_2_factory,
                       position='replace')
model.save("./temp1")
model = keras.models.load_model("./temp1")
model = insert_layer_nonseq(model=model, layer_regex='avg_pool_2', insert_layer_factory=Global_avg_pool_factory,
                       position='after')

model.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=keras.optimizers.Adam(),
    metrics=['accuracy', tf.keras.metrics.SparseTopKCategoricalAccuracy(k=5)]
)                                            
model.summary()

Evaluate

In [22]:
model.evaluate(ds_val, verbose=1)

123/123 [==============================] - 32s 246ms/step - loss: 9.5805 - accuracy: 2.5491e-04 - sparse_top_k_categorical_accuracy: 0.2027


[9.580506324768066, 0.0002549069467931986, 0.2026510387659073]

In [20]:
checkpoint_filepath = 'best_ResNet152_trick.h5'
model_checkpoint_callback = keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    monitor='"val_sparse_top_k_categorical_accuracy"',
    mode='max',
    save_best_only=True)
model.fit(ds_train, validation_data=ds_val, verbose=1, epochs=50, callbacks=[model_checkpoint_callback])

Epoch 1/50
1086/1086 [==============================] - 809s 742ms/step - loss: 6.9148 - accuracy: 0.0000e+00 - sparse_top_k_categorical_accuracy: 0.0140 - val_loss: 7.6273 - val_accuracy: 0.0000e+00 - val_sparse_top_k_categorical_accuracy: 0.0092
Epoch 2/50
1086/1086 [==============================] - 810s 744ms/step - loss: 6.6829 - accuracy: 0.0000e+00 - sparse_top_k_categorical_accuracy: 0.0211 - val_loss: 7.4928 - val_accuracy: 0.0000e+00 - val_sparse_top_k_categorical_accuracy: 0.0171
Epoch 3/50
1086/1086 [==============================] - 809s 744ms/step - loss: 6.5258 - accuracy: 0.0000e+00 - sparse_top_k_categorical_accuracy: 0.0287 - val_loss: 6.8024 - val_accuracy: 0.0000e+00 - val_sparse_top_k_categorical_accuracy: 0.0191
Epoch 4/50
1086/1086 [==============================] - 809s 744ms/step - loss: 6.4177 - accuracy: 2.8781e-05 - sparse_top_k_categorical_accuracy: 0.0333 - val_loss: 7.0978 - val_accuracy: 0.0000e+00 - val_sparse_top_k_categorical_accuracy: 0.0257
Epoch 5/

KeyboardInterrupt: 

Calculate Flops

In [ ]:
!pip install keras-flops
from keras_flops import get_flops
flops = get_flops(model, batch_size=1)
print(f"FLOPS: {flops / 10 ** 9:.03} G")

Save Model

In [21]:
Model_Name = "ResNet152_trick_35ep_41_9GF"
os.environ["MODELNAME"] = Model_Name
os.environ["MODELNAMEZIP"] = Model_Name + ".zip"
model.save("./" + Model_Name)
!zip -r ./$MODELNAMEZIP ./$MODELNAME
!rm -rf ./$MODELNAME

/opt/conda/lib/python3.7/site-packages/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


  adding: ResNet152_trick_35ep_41_9GF/ (stored 0%)
  adding: ResNet152_trick_35ep_41_9GF/saved_model.pb (deflated 93%)
  adding: ResNet152_trick_35ep_41_9GF/keras_metadata.pb (deflated 96%)
  adding: ResNet152_trick_35ep_41_9GF/variables/ (stored 0%)
  adding: ResNet152_trick_35ep_41_9GF/variables/variables.data-00000-of-00001 (deflated 7%)
  adding: ResNet152_trick_35ep_41_9GF/variables/variables.index (deflated 83%)
  adding: ResNet152_trick_35ep_41_9GF/assets/ (stored 0%)
